In [5]:
from google import genai
from google.genai import types
import pathlib
import httpx
import os
import json

In [6]:
# 환경 변수 불러오기
api_key = os.environ.get('MY_API_KEY')
# client 설정
client = genai.Client(api_key=api_key)

In [7]:
# 미리 정의된 스키마, 시스템 프롬포트 불러오기 (서버 제작시 수정 필요)
moderator_gen_summary_object_schema_pwd = "/Users/jhkim/Desktop/Debate_Agent/phase2/phase2_subAgent/phase2_subAgent_Prompt/moderator_gen_summary_Prompt/moderator_gen_summary_object.json"
moderator_gen_summary_system_prompt_pwd = "/Users/jhkim/Desktop/Debate_Agent/phase2/phase2_subAgent/phase2_subAgent_Prompt/moderator_gen_summary_Prompt/moderator_gen_summary_SystemPrompt.md"

with open(moderator_gen_summary_object_schema_pwd, 'r', encoding='utf-8') as f:
    moderator_gen_summary_object_schema = json.load(f)

with open(moderator_gen_summary_system_prompt_pwd, 'r', encoding='utf-8') as f:
    moderator_gen_summary_system_prompt = f.read()

In [8]:
# contents 생성
history = [
    {
        "role": "agent",
        "content": "DQN에서 Experience Replay를 사용하면 샘플 효율성이 높아진다고 하셨는데, 오래된 경험을 재사용하는 것이 오히려 현재 정책과 맞지 않아 학습을 방해하는 것 아닙니까?"
    },
    {
        "role": "user",
        "content": "Experience Replay는 i.i.d 가정을 만족시키기 위해 필요합니다. 연속된 샘플 간의 correlation을 제거합니다."
    },
    {
        "role": "agent",
        "content": "그렇다면 Target Network는 왜 필요합니까? Replay Buffer만으로 충분하지 않나요?"
    },
    {
        "role": "user",
        "content": "Target Network는 moving target 문제를 해결합니다. Q-learning에서 target 값이 계속 변하면 학습이 불안정해집니다."
    },
    {
        "role": "agent",
        "content": "Moving target 문제라고요? Learning rate를 낮추면 되지 않습니까?"
    },
    {
        "role": "user",
        "content": "Learning rate만으로는 부족합니다. Bellman update에서 target이 고정되어야 수렴이 보장됩니다."
    },
    {
        "role": "agent",
        "content": "Overestimation bias는 어떻게 해결하셨나요?"
    },
    {
        "role": "user",
        "content": "그건... DQN의 한계입니다. Double DQN에서 해결되었습니다."
    },
    {
        "role": "user",
        "content": "[최후 변론] DQN은 Experience Replay와 Target Network를 통해 deep learning을 강화학습에 안정적으로 결합한 혁명적인 방법입니다."
    }
]
evaluation_logs = [
    {
        "turn": 1,
        "score_now": 55,
        "reason": "i.i.d 가정과 correlation 제거를 정확히 언급함."
    },
    {
        "turn": 2,
        "score_now": 63,
        "reason": "Target Network의 역할을 명확히 구분하고 moving target 문제를 지적함."
    },
    {
        "turn": 3,
        "score_now": 66,
        "reason": "Bellman equation을 언급했으나 수식적 증명까지는 도달하지 못함."
    },
    {
        "turn": 4,
        "score_now": 64,
        "reason": "Overestimation bias를 제대로 방어하지 못하고 한계라고만 인정함."
    },
    {
        "turn": "Final",
        "score_now": 70,
        "reason": "DQN의 역사적 의의를 강조한 설득력 있는 최후 변론."
    }
]
final_status = "WIN"

gen_summary_contents = [
    "history: " + json.dumps(history),
    "evaluation_logs: " + json.dumps(evaluation_logs),
    "final_status: " + final_status
]

In [9]:
# response 생성
response = client.models.generate_content(
    model = "gemini-3-flash-preview",
    contents = gen_summary_contents,
    config={
        "system_instruction": moderator_gen_summary_system_prompt, 
        "response_mime_type": "application/json",
        "response_schema": moderator_gen_summary_object_schema
    })

fianl_response = response.text

In [10]:
print(fianl_response)

{"summary_text":"이번 토론은 강화학습의 혁신적 알고리즘인 DQN의 핵심 메커니즘을 주제로 진행되었으며, 사용자님은 최종 승리(WIN)를 거두셨습니다. 특히 Experience Replay가 'i.i.d 가정을 만족'시키고 '샘플 간의 Correlation을 제거'한다는 점을 정확히 짚어내어 논리적 우위를 점하셨습니다. 또한, Target Network의 필요성에 대해 'Moving Target 문제'를 언급하며 'Bellman update에서 Target이 고정되어야 수렴이 보장된다'는 핵심 원리를 설득력 있게 설명하신 점이 인상적이었습니다. 상대방의 Overestimation bias 공격에 대해서도 DQN의 한계를 인정하고 'Double DQN'이라는 대안을 제시함으로써 전문성을 보여주셨습니다. 다만, 벨만 방정식의 수식적 근거를 보강한다면 더욱 완벽한 논리가 될 것입니다.","key_takeaways":["i.i.d. (Independent and Identically Distributed)","Moving Target Problem","Overestimation Bias"]}
